<a href="https://colab.research.google.com/github/solobala/numpy36/blob/main/Balakina_numpy36_DZ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание «Библиотека Pandas»

In [ ]:
import pandas as pd

# Задание 1
Скачайте с сайта grouplens.org...movielens/ датасет любого размера. Определите, какому фильму было выставлено больше всего оценок 5.0.

In [ ]:
!wget -O "ml-latest-small.zip" "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"   # Качаем архив выбранного датасета

--2022-09-14 08:57:38--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  1.52MB/s    in 0.6s    

2022-09-14 08:57:39 (1.52 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [ ]:
from zipfile import ZipFile  # Подключаем библиотеку ZipFile из пакета zipfile

In [ ]:
with ZipFile('ml-latest-small.zip', 'r') as zip_file:  # Распаковываем архив в сессионное хранилище в папку ml-latest-small. Нам понадобятся файлы ratings.csv и movies.csv
    # print(zip_file.infolist())
    zip_file.extract('ml-latest-small/ratings.csv')
    zip_file.extract('ml-latest-small/movies.csv')

In [ ]:
data_ratings = pd.read_csv('/content/ml-latest-small/ratings.csv') # Читаем файл raiting.csv

In [ ]:
data_ratings_filtered = data_ratings[data_ratings['rating'] == 5.0]  # отфильтровываем фильмы с оценками 5.0  в новый датасет и далее работаем уже с ним

In [ ]:
# Ищем идентификатор фильма с наибольшим к-вом пятерок. 
# для этого группируем фильмы по movieId, 
# считаем кол-во пятерок для каждого идентификаторы, 
# сортируем по убыванию и берем только 1-ю строку
%%time

my_filter = data_ratings_filtered[['movieId', 'rating']].groupby('movieId').count().sort_values(by = 'rating', ascending = False).head(1).index.values[0]  # находим значение фильтра - способ 1
# type(data_ratings_filtered[['movieId', 'rating']].groupby('movieId').count().sort_values(by = 'rating', ascending = False).head(1)) # Тип результата до использования index.values[0] - DataFrame
my_filter

CPU times: user 7.8 ms, sys: 57 µs, total: 7.85 ms
Wall time: 11.1 ms


318

In [ ]:
%%time
my_filter = data_ratings_filtered.movieId.value_counts().head(1).index.values[0]  #  Находим значение фильтра - способ 2. Но тогда, Merge использовать нельзя т.к получается не датафрейм, а серия. Но ищет быстрее в 3 раза
my_filter
# type(data_ratings_filtered.movieId.value_counts().head(1)) #Тип - Series


CPU times: user 2.5 ms, sys: 0 ns, total: 2.5 ms
Wall time: 2.49 ms


318

In [ ]:
data = pd.read_csv('/content/ml-latest-small/movies.csv') # теперь читаем файл movies.csv с названиями фильмов

In [ ]:
%%time
print(f"Фильм с наибольшим количеством оценок 5.0: {data[data['movieId'] == my_filter]['title'].values[0]}") # подставляем сюда как  значение фильтра - my_filter

Фильм с наибольшим количеством оценок 5.0: Shawshank Redemption, The (1994)
CPU times: user 3.12 ms, sys: 1.04 ms, total: 4.16 ms
Wall time: 4.15 ms


In [ ]:
%%time
# можно сразу подставить значение фильтра,найденное способом 2, не обязательно помещать его в переменную My_filter - выходит в 6 раз быстрее чем по способу 1
print(f"Фильм с наибольшим количеством оценок 5.0: {data[data['movieId'] == data_ratings_filtered.movieId.value_counts().head(1).index.values[0]]['title'].values[0]}")

Фильм с наибольшим количеством оценок 5.0: Shawshank Redemption, The (1994)
CPU times: user 2.68 ms, sys: 51 µs, total: 2.73 ms
Wall time: 2.75 ms


In [ ]:
%%time
# можно сразу подставить значение фильтра, найденное  сособом 1, не обязательно помещать его в переменную My_filter
print(f"Фильм с наибольшим количеством оценок 5.0: {data[data['movieId'] == data_ratings_filtered[['movieId', 'rating']].groupby('movieId').count().sort_values(by = 'rating', ascending = False).head(1).index.values[0]]['title'].values[0]}")

Фильм с наибольшим количеством оценок 5.0: Shawshank Redemption, The (1994)
CPU times: user 9.83 ms, sys: 0 ns, total: 9.83 ms
Wall time: 11.2 ms


In [ ]:
%%time
# попытка решитьс  помощью merge . выходит очень долго, и сипользовать можно только значенние my_filter по способу 1, т.к с Series нельзя делать merge
joined = data_ratings_filtered[['movieId', 'rating']].groupby('movieId').count().sort_values(by = 'rating', ascending = False).head(1).merge(data, on ='movieId', how = 'left') # другой способ - решение через merge, затрачено в 7 раз больше времени


CPU times: user 11.5 ms, sys: 116 µs, total: 11.6 ms
Wall time: 13.6 ms


In [ ]:
%%time
print(f"Фильм с наибольшим количеством оценок 5.0: {joined['title'].values[0]}")

Фильм с наибольшим количеством оценок 5.0: Shawshank Redemption, The (1994)
CPU times: user 1.59 ms, sys: 0 ns, total: 1.59 ms
Wall time: 1.61 ms


# Задание 2
По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.

In [25]:
!wget -O "10._%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_pandas.zip" "https://u.netology.ngcdn.ru/backend/uploads/lms/attachments/files/data/15255/10._%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_pandas.zip"

--2022-09-14 09:40:02--  https://u.netology.ngcdn.ru/backend/uploads/lms/attachments/files/data/15255/10._%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_pandas.zip
Resolving u.netology.ngcdn.ru (u.netology.ngcdn.ru)... 104.20.157.54, 104.20.156.54, 2606:4700:10::6814:9c36, ...
Connecting to u.netology.ngcdn.ru (u.netology.ngcdn.ru)|104.20.157.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4762826 (4.5M) [application/x-zip-compressed]
Saving to: ‘10._%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_pandas.zip’

10._%D0%9E%D1%81%D0 100%[===================>]   4.54M  15.5MB/s    in 0.3s    

2022-09-14 09:40:03 (15.5 MB/s) - ‘10._%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_pandas.zip’ saved [4762826/4762826]



In [26]:
with ZipFile("10._%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_pandas.zip", 'r') as zip_file:  # Распаковываем архив в сессионное хранилище 
   # print(zip_file.infolist())
    zip_file.extract('power.csv')

In [27]:
data = pd.read_csv('/content/power.csv')
data.head()

,country,year,quantity,category
0,Austria,1996,5.0,1
1,Austria,1995,17.0,1
2,Belgium,2014,0.0,1
3,Belgium,2013,0.0,1
4,Belgium,2012,35.0,1


##Вариант 1. Решение с помощью функций

In [ ]:
%%time
def my_filter(row): #быстрее ищет по множеству, а не по списку, но здесь маленькая длина списка, нет смысла
    """Отбор сразу по всем критериям - вхождению в Прибалтику, 
    нужной категории, году и неотрицатиельному количеству"""
    return row['country'] in ['lithuania', 'Latvia', 'Estonia'] \
      and  row['category'] in [4, 12, 21] \
      and row['year'] in range(2005,2011) \
      and row['quantity'] >= 0
    

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 10 µs


In [ ]:
def my_categories(category):
    """ Отбор категорий потребления согласно критерию 
    не использовать, если используем
      my_filter
    """
    if category in [4, 12, 21]:
      return "selected"
    return "Other"

In [ ]:
def baltic(country):
    """Объединение стран Прибалтики
      не использовать, если используем
      my_filter
    """
    if country in ['lithuania', 'Latvia', 'Estonia']:
      return 'Прибалтика'
    return 'Other'

In [ ]:
%%time
data['my_categories'] = data.category.apply(my_categories) #не исп., если исп. my_filter
data['my_categories']


CPU times: user 218 ms, sys: 1.6 ms, total: 219 ms
Wall time: 219 ms


0          Other
1          Other
2          Other
3          Other
4          Other
           ...  
1189477    Other
1189478    Other
1189479    Other
1189480    Other
1189481    Other
Name: my_categories, Length: 1189482, dtype: object

In [ ]:
%%time
data['baltic'] = data.country.apply(baltic) #не исп., если исп. my_filter

CPU times: user 222 ms, sys: 2.7 ms, total: 225 ms
Wall time: 232 ms


In [ ]:
%%time
filtered_data = data [(data['baltic'] == 'Прибалтика')  #не исп., если исп. my_filter
& (data['my_categories'] == 'selected') 
& (data['year'] > 2004)  
& (data['year'] < 2011) 
& (data['quantity']>=0)]
print(f" суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: {sum(filtered_data['quantity'])}")


 суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: 118284.0
CPU times: user 175 ms, sys: 651 µs, total: 175 ms
Wall time: 171 ms


In [ ]:
%%time
data['my_filter'] = data.apply(my_filter, axis = 1)
filtered_data = data [(data['my_filter'] == True)] 
print(f" суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: {sum(filtered_data['quantity'])}")
# Очень долго!

 суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: 118284.0
CPU times: user 11 s, sys: 66 ms, total: 11.1 s
Wall time: 11.1 s


##Вариант 2. Решение с использованием lambda - функции

In [ ]:
%%time
data['my_filter'] = data.apply(lambda x: x['country'] in ['lithuania', 'Latvia', 'Estonia'] \
      and x['category'] in [4, 12, 21] \
      and x['year'] in range(2005,2011) \
      and x['quantity'] >= 0 ,axis = 1)
# Очень долго!

CPU times: user 11.2 s, sys: 96.3 ms, total: 11.3 s
Wall time: 11.4 s


In [ ]:
%%time
filtered_data = data [(data['my_filter'] == True)] 
print(f" суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: {sum(filtered_data['quantity'])}")

 суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: 118284.0
CPU times: user 5.91 ms, sys: 0 ns, total: 5.91 ms
Wall time: 4.74 ms


##Вариант 3. Решение через набор фильтров и query - самый быстрый способ

In [ ]:
%%time
filtered_columns = data.filter(items =['country','year','quantity', 'category']) # Отбираем столбцы, по которым будем фильтровать

CPU times: user 24.8 ms, sys: 894 µs, total: 25.7 ms
Wall time: 30.6 ms


In [ ]:
%%time
filtered_data = filtered_columns.query("(quantity >= 0) & (year >= 2005) & (year <= 2010) & category in [4,12,21] and country in  ['lithuania', 'Latvia', 'Estonia']")
print(f" суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: {sum(filtered_data['quantity'])}")

 суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: 118284.0
CPU times: user 86 ms, sys: 0 ns, total: 86 ms
Wall time: 77.5 ms


##Вариант 4. Решение через набор фильтров

In [ ]:
%%time
filtered_data = data[((data['country'] == 'lithuania') | (data['country'] =='Latvia') | (data['country'] =='Estonia')) & (data['quantity'] >= 0) & ((data['category'] == 4) | (data['category'] == 12) | (data['category'] == 21)) & ((data['year'] >= 2005) & (data['year'] <= 2010)) ] 
print(f" суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: {sum(filtered_data['quantity'])}")



 суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: 118284.0
CPU times: user 254 ms, sys: 3.85 ms, total: 258 ms
Wall time: 251 ms


## Вариант 5. Решение через набор фильтров и метод .isin() - в 3 раза быстрее по ср. с вариантом 4 + меньше кода писать

In [28]:
%%time
filtered_data = data[
    (data['country'].isin(['lithuania','Latvia','Estonia'])) & 
    (data['quantity'] >= 0) & 
    (data['category'].isin([4,12,21])) & 
    (data['year'].isin([2005,2006,2007,2008,2009,2010]))
    ] 
print(f" суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: {sum(filtered_data['quantity'])}")


 суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: 118284.0
CPU times: user 78.1 ms, sys: 2.33 ms, total: 80.5 ms
Wall time: 79.8 ms


# Задание 3
Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.
Примеры страниц (необязательно брать именно эти):
https://fortrader.org/quotes
www.finanz.ru...om-vremeni

In [ ]:
page_url = 'https://www.finanz.ru/aktsii/polymetal'

In [ ]:
df = pd.read_html(page_url, attrs ={'class':'header_user_pricebox'}, encoding = 'utf-8')

In [ ]:
df

[     0       1     2        3       4      5        6       7      8   \
 0  ММВБ   2 137  2,3%   Nasdaq  12 494  -1,2%  Биткойн  22 883  -1,3%   
 1   РТС   1 116  3,0%   S&P500   4 122  -0,4%    Нефть     961  -0,5%   
 2   Dow  32 833  0,1%  FTSE100   7 488   0,1%   Золото   1 794   0,0%   
 
         9       10    11  
 0  USD/RUB  604750  0,0%  
 1  EUR/RUB  617903  0,0%  
 2  EUR/USD   10218  0,0%  ]